In [1]:
from kafka import KafkaConsumer
from json import loads
from cassandra.cluster import Cluster
import pandas as pd

In [2]:
consumer = KafkaConsumer(
    'Amazon_Stock_Daily_Data',
     bootstrap_servers=['localhost:9092'], 
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [ ]:
#for con in consumer:
#    print(con.value)

In [3]:
# Connect to Cassandra
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [ ]:
# Create Keyspace
session.execute("""
CREATE KEYSPACE IF NOT EXISTS stellar
  WITH REPLICATION = { 
   'class' : 'SimpleStrategy', 
   'replication_factor' : 3
  };
""")

In [5]:
# Check Keyspace
rows = session.execute("SELECT keyspace_name FROM system_schema.keyspaces;")
for row in rows:
    print(row.keyspace_name)

system_auth
system_schema
stellar
system_distributed
system
system_traces


In [6]:
# Use Keyspace
session.set_keyspace('stellar')

In [106]:
# Query to drop table
#query = f"DROP TABLE stellar.Amazon_Stock_Daily"

# Execute query
#session.execute(query)

In [ ]:
# create table
session.execute("""
CREATE TABLE IF NOT EXISTS Amazon_Stock_Daily (
                Date date,
                Open decimal,
                High decimal,
                Low decimal,
                Close decimal,
                Volume decimal,
                PRIMARY KEY (Date)
            );
""")

In [8]:
insert_query = session.prepare(
    f"INSERT INTO Amazon_Stock_Daily (date, open, high, low, close, volume) VALUES (?, ?, ?, ?, ?, ?)"
)

In [9]:
# insert data
for message in consumer:
    data = message.value  # Information received from KafkaConsumer
    session.execute(insert_query, (
        data['Date'],
        data['Open'],
        data['High'],
        data['Low'],
        data['Close'],
        data['Volume']
    ))
    print(f"Inserted data for Date: {data['Date']}")

Inserted data for Date: 2006-08-03
Inserted data for Date: 2023-12-14
Inserted data for Date: 2009-04-17
Inserted data for Date: 2012-10-17
Inserted data for Date: 2020-07-30
Inserted data for Date: 1999-10-22
Inserted data for Date: 1999-04-23
Inserted data for Date: 2020-11-23
Inserted data for Date: 2004-03-12
Inserted data for Date: 2022-01-03
Inserted data for Date: 2003-12-31
Inserted data for Date: 2003-11-21
Inserted data for Date: 2013-04-05
Inserted data for Date: 2022-12-30
Inserted data for Date: 2011-08-04


In [111]:
# Check data
#r = session.execute("SELECT * FROM Amazon_Stock_Daily")
#df = pd.DataFrame(r)
#df

,date,close,high,low,open,volume
0,2020-09-18,147.7454986572265625,151.8899993896484375,145.2769927978515625,151.587005615234375,177852000
1,2014-12-01,16.299999237060546875,17.0319995880126953125,16.2964992523193359375,16.9060001373291015625,98898000
2,2002-05-07,0.805499970912933349609375,0.81749999523162841796875,0.801500022411346435546875,0.805499970912933349609375,110874000
3,2000-07-27,1.56875002384185791015625,1.60000002384185791015625,1.4874999523162841796875,1.57187497615814208984375,471586000
4,2010-09-22,7.59149980545043856494658029987476766109466552...,7.6350002288818359375,7.4549999237060546875,7.492000102996826171875,132248000
...,...,...,...,...,...,...
684,2006-11-09,1.94200003147125244140625,1.98849999904632568359375,1.9405000209808349609375,1.97500002384185791015625,116142000
685,2009-12-08,6.705500125885009765625,6.80399990081787109375,6.643499851226806640625,6.715000152587890625,160056000
686,2006-02-06,1.89750003814697287829460492503130808472633361...,1.9170000553131103515625,1.88849997520446799548210492503130808472633361...,1.9170000553131103515625,170064000
687,2021-12-27,169.66949462890625,172.9429931640625,169.2154998779296875,171.0370025634765625,58688000
